In [1]:
text = ""

with open("./input.txt", "r") as file:
	text = file.read()
	

print("Total chars: ", len(text))

Total chars:  1115394


In [2]:
vocabulary_list = sorted(list(set(text)))

print("Vocabulary: ", "".join(vocabulary_list))
print("Lenght of vocabulary: ", len(vocabulary_list))

Vocabulary:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Lenght of vocabulary:  65


In [3]:
stoi = { char: i for i, char in enumerate(vocabulary_list) }
itos = { i: char for i, char in enumerate(vocabulary_list) }

encode = lambda s: [stoi[char] for char in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("test encoding"))
print(decode(encode("test encoding")))

[58, 43, 57, 58, 1, 43, 52, 41, 53, 42, 47, 52, 45]
test encoding


In [4]:
import torch

In [5]:
tensor = torch.tensor(encode(text), dtype=torch.long)
print(tensor.shape, tensor.dtype)
print(tensor[:1000])

n = int(0.9 * len(tensor))
train_data, val_data = tensor[:n], tensor[n:]

block_size = 8
print(train_data[:block_size+1])


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
	context = x[:t + 1]
	target = y[t]

	print(f"when input is {context} then target is {target}")

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
	data = train_data if split == "train" else val_data
	ix = torch.randint(len(data) - block_size, (batch_size,))
	print("ix", ix)

	x = torch.stack([data[i:i + block_size] for i in ix])
	y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

	return x, y

xb, yb = get_batch("train")
print("inputs:")

print(xb.shape)
print(xb)

print("targets:")
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
	for t in range(block_size):
		context = xb[b, :t + 1]
		target = yb[b, t]

		print(f"when input is {context} then target is {target}")

ix tensor([385543, 765902, 890262, 369401])
inputs:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 51,  1, 39,  1, 40],
        [56, 43,  6,  1, 52, 53, 56,  1],
        [52,  1, 58, 46, 39, 58,  1, 54],
        [46, 47, 57,  1, 51, 43, 52,  1]])
targets:
torch.Size([4, 8])
tensor([[46, 47, 51,  1, 39,  1, 40, 56],
        [43,  6,  1, 52, 53, 56,  1, 53],
        [ 1, 58, 46, 39, 58,  1, 54, 39],
        [47, 57,  1, 51, 43, 52,  1, 53]])
------
when input is tensor([1]) then target is 46
when input is tensor([ 1, 46]) then target is 47
when input is tensor([ 1, 46, 47]) then target is 51
when input is tensor([ 1, 46, 47, 51]) then target is 1
when input is tensor([ 1, 46, 47, 51,  1]) then target is 39
when input is tensor([ 1, 46, 47, 51,  1, 39]) then target is 1
when input is tensor([ 1, 46, 47, 51,  1, 39,  1]) then target is 40
when input is tensor([ 1, 46, 47, 51,  1, 39,  1, 40]) then target is 56
when input is tensor([56]) then target is 43
when input is tensor([56, 43]) then target

In [18]:
a = torch.randint(4, (4,8))

In [28]:
a


tensor([[2, 0, 3, 2, 3, 0, 0, 0],
        [2, 3, 3, 2, 3, 1, 1, 1],
        [3, 2, 1, 0, 3, 3, 2, 2],
        [1, 2, 2, 0, 1, 2, 1, 2]])

In [29]:
a[3]

tensor([1, 2, 2, 0, 1, 2, 1, 2])

In [31]:
a[3, :3]

tensor([1, 2, 2])